# Reconnaissance d'images avec TensorFlow
Ce notebook décrit de la reconnaissance d'images simples avec un modèle de CNN sur Tensorflow.
Il est inspiré d'un [Tutorial Tensorflow](https://www.tensorflow.org/tutorials/layers).

Le but est d'apprendre à reconnaitre des chiffres écrits à la main sous forme d'image mono-chrome (de taille 28x28).
Il s'agit des données [MNIST](http://yann.lecun.com/exdb/mnist/)

## Exemples:
<img align="left" src="mnist_plot.png" width="400">


## Import des librairies
En particulier Tensorflow

In [12]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

## Définition du modele
Il est basé sur un modèle d'apprentissage profond avec plusieurs couches:
* Couche d'entrée (28x28x1)
* Couches de Convolution (CNN)
* Couches de Max_pooling
* Couche de réseau de neurones dense 
* Couche de sortie correspondant aux 10 chiffres (0 à 9) à classifier

In [13]:
tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## Chargement des données d'apprentissage

In [14]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz


In [15]:
print("Train data:",train_data.shape)
print("Test data:",eval_data.shape)

Train data: (55000, 784)
Test data: (10000, 784)


## Création et entrainement de l'estimateur

In [16]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir=".")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '.', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C32ADD3E48>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=200)

In [18]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from .\model.ckpt-2200
INFO:tensorflow:Saving checkpoints for 2201 into .\model.ckpt.
INFO:tensorflow:loss = 0.7134538, step = 2201
INFO:tensorflow:global_step/sec: 20.1297
INFO:tensorflow:loss = 0.84040284, step = 2301 (4.969 sec)
INFO:tensorflow:global_step/sec: 20.3388
INFO:tensorflow:loss = 0.5345657, step = 2401 (4.917 sec)
INFO:tensorflow:global_step/sec: 20.3897
INFO:tensorflow:loss = 0.54208016, step = 2501 (4.905 sec)
INFO:tensorflow:global_step/sec: 20.2762
INFO:tensorflow:loss = 0.567592, step = 2601 (4.931 sec)
INFO:tensorflow:global_step/sec: 20.3195
INFO:tensorflow:loss = 0.40071493, step = 2701 (4.921 sec)
INFO:tensorflow:global_step/sec: 20.3691
INFO:tensorflow:loss = 0.4642006, step = 2801 (4.909 sec)
INFO:tensorflow:global_step/sec: 20.2501
INFO:tensorflow:loss = 0.52017635, step = 2901 (4.938 sec)
INFO:tensorflow:global_step/sec: 20.3346
INFO:tensorflow:loss = 0.60854876, step = 3001 (4.

## Evaluation sur les données de test
On évalue le modèle préalablement entrainé sur les données de test pour verifier la justesse de la prédiction.

In [19]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print("Résultats de l'évaluation:",eval_results)  

INFO:tensorflow:Starting evaluation at 2018-02-19-16:22:07
INFO:tensorflow:Restoring parameters from .\model.ckpt-4200
INFO:tensorflow:Finished evaluation at 2018-02-19-16:22:09
INFO:tensorflow:Saving dict for global step 4200: accuracy = 0.918, global_step = 4200, loss = 0.2997258
Résultats de l'évaluation: {'accuracy': 0.918, 'loss': 0.2997258, 'global_step': 4200}


## Conclusion
L'exactitude des prédictions dépend du nombre d'itérations lors de l'apprentissage 
(on utilise la méthode du gradient stochastique pour optimiser les paramètres du modèle).

Avec un peu de patience, on peut atteindre une precision de plus de 97%!